In [1]:
%matplotlib inline


Save and Load the Model
============================

In this section we will look at how to persist model state with saving, loading and running model predictions.



In [2]:
import torch
import torch.onnx as onnx
import torchvision.models as models

Saving and Loading Model Weights
--------------------------------
PyTorch models store the learned parameters in an internal
state dictionary, called ``state_dict``. These can be persisted via the ``torch.save``
method:



In [3]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


To load model weights, you need to create an instance of the same model first, and then load the parameters 
using ``load_state_dict()`` method.



In [4]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

<div class="alert alert-info"><h4>Note</h4><p>be sure to call ``model.eval()`` method before inferencing to set the dropout and batch normalization layers to evaluation mode. Failing to do this will yield inconsistent inference results.</p></div>



Saving and Loading Models with Shapes
-------------------------------------
When loading model weights, we needed to instantiate the model class first, because the class 
defines the structure of a network. We might want to save the structure of this class together with 
the model, in which case we can pass ``model`` (and not ``model.state_dict()``) to the saving function:



In [6]:
torch.save(model, 'model.pth')

We can then load the model like this:



In [7]:
model = torch.load('model.pth')

<div class="alert alert-info"><h4>Note</h4><p>This approach uses Python `pickle <https://docs.python.org/3/library/pickle.html>`_ module when serializing the model, thus it relies on the actual class definition to be available when loading the model.</p></div>



Exporting Model to ONNX
-----------------------
PyTorch also has native ONNX export support. Given the dynamic nature of the
PyTorch execution graph, however, the export process must
traverse the execution graph to produce a persisted ONNX model. For this reason, a
test variable of the appropriate size should be passed in to the
export routine (in our case, we will create a dummy zero tensor of the correct size):



In [8]:
input_image = torch.zeros((1,3,224,224))
onnx.export(model, input_image, 'model.onnx')